In [2]:
# Import packages

import pandas as pd 
import numpy as np 
import os

pd.set_option("display.max_columns", None)

In [3]:
betting_df = pd.read_excel('https://www.aussportsbetting.com/historical_data/nfl.xlsx')

<ipython-input-3-41d42a54ed34>:1: FutureWarning: Inferring datetime64[ns] from data containing strings is deprecated and will be removed in a future version. To retain the old behavior explicitly pass Series(data, dtype=datetime64[ns])
  betting_df = pd.read_excel('https://www.aussportsbetting.com/historical_data/nfl.xlsx')


In [4]:
betting_df.head()

,Date,Home Team,Away Team,Home Score,Away Score,Overtime?,Playoff Game?,Neutral Venue?,Home Odds Open,Home Odds Min,Home Odds Max,Home Odds Close,Away Odds Open,Away Odds Min,Away Odds Max,Away Odds Close,Home Line Open,Home Line Min,Home Line Max,Home Line Close,Away Line Open,Away Line Min,Away Line Max,Away Line Close,Home Line Odds Open,Home Line Odds Min,Home Line Odds Max,Home Line Odds Close,Away Line Odds Open,Away Line Odds Min,Away Line Odds Max,Away Line Odds Close,Total Score Open,Total Score Min,Total Score Max,Total Score Close,Total Score Over Open,Total Score Over Min,Total Score Over Max,Total Score Over Close,Total Score Under Open,Total Score Under Min,Total Score Under Max,Total Score Under Close,Notes
0,2022-11-07,New Orleans Saints,Baltimore Ravens,13,27,NaN,NaN,NaN,2.45,2.05,2.65,2.10,1.60,1.54,1.80,1.76,3.5,1.5,4.0,1.5,-3.5,-4.0,-1.5,-1.5,1.9,1.90,1.9,1.95,1.9,1.90,1.90,1.86,49.0,46.0,49.0,46.0,1.9,1.9,1.9,1.9,1.9,1.9,1.9,1.9,NaN
1,2022-11-06,Kansas City Chiefs,Tennessee Titans,20,17,Y,NaN,NaN,1.18,1.11,1.18,1.11,5.25,5.25,6.75,6.75,-10.5,-14.0,-10.5,-14.0,10.5,10.5,14.0,14.0,1.9,1.90,1.9,1.90,1.9,1.90,1.90,1.90,47.0,45.0,47.0,45.0,1.9,1.9,1.9,1.9,1.9,1.9,1.9,1.9,NaN
2,2022-11-06,Tampa Bay Buccaneers,Los Angeles Rams,16,13,NaN,NaN,NaN,1.80,1.60,1.80,1.64,2.05,2.05,2.45,2.35,-1.5,-3.0,-1.5,-3.0,1.5,1.5,3.0,3.0,1.9,1.90,1.9,1.95,1.9,1.90,1.90,1.86,43.0,42.0,43.0,42.5,1.9,1.9,1.9,1.9,1.9,1.9,1.9,1.9,NaN
3,2022-11-06,Arizona Cardinals,Seattle Seahawks,21,31,NaN,NaN,NaN,1.52,1.52,1.80,1.76,2.67,2.05,2.67,2.10,-3.5,-3.5,-1.5,-2.0,3.5,1.5,3.5,2.0,1.9,1.90,1.9,1.90,1.9,1.90,1.90,1.90,49.0,49.0,50.5,49.0,1.9,1.9,1.9,1.9,1.9,1.9,1.9,1.9,NaN
4,2022-11-06,Atlanta Falcons,Los Angeles Chargers,17,20,NaN,NaN,NaN,2.40,2.25,2.45,2.25,1.62,1.60,1.68,1.68,3.0,2.5,3.0,2.5,-3.0,-3.0,-2.5,-2.5,1.9,1.95,2.0,1.95,1.9,1.83,1.86,1.86,47.0,47.0,49.5,49.5,1.9,1.9,1.9,1.9,1.9,1.9,1.9,1.9,NaN


In [5]:
betting_data = betting_df.copy()
betting_data = betting_data[['Date', 'Home Team', 'Away Team', 'Home Score', 'Away Score', 'Home Odds Open', 'Away Odds Open',
                             'Home Line Open', 'Away Line Open', 'Total Score Open','Home Odds Close', 'Away Odds Close', 
                             'Home Line Close', 'Away Line Close', 'Total Score Close']]

In [7]:
def decimal_to_moneyline(decimal):
    if np.isnan(decimal):
        return np.nan
    elif decimal >= 2.00:
        return np.round((decimal-1) * 100)
    else: 
        return np.round(-100 / (decimal-1))
    
def get_over_under_result(game, lineopen=True):
    game_score = game['Total Score']
    if lineopen:
        ou_score = game['Total Score Open']
    else:
        ou_score = game['Total Score Close']
        
    if game_score == ou_score :
        result = 'push'
    elif game_score > ou_score:
        result = 'over'
    else:
        result = 'under'
        
    return result 

def fix_home_team_names(game):
    team_mapping = {
        'Oakland Raiders':'Las Vegas Raiders',
        'St. Louis Rams':'Los Angeles Rams',
        'San Diego Chargers':'Los Angeles Chargers',
        'Washington Redskins':'Washington Football Team',
        'Washington Commanders':'Washington Football Team'
    }
    
    if game['Home Team'] in team_mapping.keys():
        return team_mapping[game['Home Team']]
    
    else:
        return game['Home Team']
    
def fix_away_team_names(game):
    team_mapping = {
        'Oakland Raiders':'Las Vegas Raiders',
        'St. Louis Rams':'Los Angeles Rams',
        'San Diego Chargers':'Los Angeles Chargers',
        'Washington Redskins':'Washington Football Team',
        'Washington Commanders':'Washington Football Team'
    }
    
    if game['Away Team'] in team_mapping.keys():
        return team_mapping[game['Away Team']]
    
    else:
        return game['Away Team']
    
def get_spread_cover_team(game, lineopen = True):
    if lineopen:
        if game['Home Team Actual Line'] < game['Home Line Open']:
            return 'home'
        elif game['Home Team Actual Line'] > game['Home Line Open']:
            return 'away'
        elif game['Home Team Actual Line'] == game['Home Line Open']:
            return 'push'
        
    else:
        if game['Home Team Actual Line'] < game['Home Line Close']:
            return 'home'
        elif game['Home Team Actual Line'] > game['Home Line Close']:
            return 'away'
        elif game['Home Team Actual Line'] == game['Home Line Close']:
            return 'push'
        
        
def get_winner(game, lineopen = True):
    if game['Home Score'] > game['Away Score']:
        return 'home'
    elif game['Home Score'] < game['Away Score']:
        return 'away'
    elif game['Home Score'] == game['Away Score']:
        return 'push'


betting_data['Home Team'] = betting_data.apply(lambda x: fix_home_team_names(x), axis=1)
betting_data['Away Team'] = betting_data.apply(lambda x: fix_away_team_names(x), axis=1)

betting_data['Home ML Open'] = betting_data['Home Odds Open'].apply(lambda x: decimal_to_moneyline(x))
betting_data['Away ML Open'] = betting_data['Away Odds Open'].apply(lambda x: decimal_to_moneyline(x))

betting_data['Home ML Close'] = betting_data['Home Odds Close'].apply(lambda x: decimal_to_moneyline(x))
betting_data['Away ML Close'] = betting_data['Away Odds Close'].apply(lambda x: decimal_to_moneyline(x))

betting_data['Total Score'] = betting_data.apply(lambda x: x['Home Score'] + x['Away Score'], axis=1)

betting_data['Over/Under Open Result'] = betting_data.apply(lambda x: get_over_under_result(x, lineopen=True), axis=1)
betting_data['Over/Under Close Result'] = betting_data.apply(lambda x: get_over_under_result(x, lineopen=False), axis=1)

betting_data['Home Team Actual Line'] = betting_data.apply(lambda x: x['Away Score'] - x['Home Score'], axis=1)
betting_data['Away Team Actual Line'] = betting_data.apply(lambda x: x['Home Score'] - x['Away Score'], axis=1)

betting_data['Spread Cover Open Result'] = betting_data.apply(lambda x: get_spread_cover_team(x, lineopen=True), axis=1)
betting_data['Spread Cover Close Result'] = betting_data.apply(lambda x: get_spread_cover_team(x, lineopen=False), axis=1)

betting_data['Game Winner'] = betting_data.apply(lambda x: get_winner(x, lineopen=True), axis=1)

betting_data.head(5)

,Date,Home Team,Away Team,Home Score,Away Score,Home Odds Open,Away Odds Open,Home Line Open,Away Line Open,Total Score Open,Home Odds Close,Away Odds Close,Home Line Close,Away Line Close,Total Score Close,Home ML Open,Away ML Open,Home ML Close,Away ML Close,Total Score,Over/Under Open Result,Over/Under Close Result,Home Team Actual Line,Away Team Actual Line,Spread Cover Open Result,Spread Cover Close Result,Game Winner
0,2022-11-07,New Orleans Saints,Baltimore Ravens,13,27,2.45,1.60,3.5,-3.5,49.0,2.10,1.76,1.5,-1.5,46.0,145.0,-167.0,110.0,-132.0,40,under,under,14,-14,away,away,away
1,2022-11-06,Kansas City Chiefs,Tennessee Titans,20,17,1.18,5.25,-10.5,10.5,47.0,1.11,6.75,-14.0,14.0,45.0,-556.0,425.0,-909.0,575.0,37,under,under,-3,3,away,away,home
2,2022-11-06,Tampa Bay Buccaneers,Los Angeles Rams,16,13,1.80,2.05,-1.5,1.5,43.0,1.64,2.35,-3.0,3.0,42.5,-125.0,105.0,-156.0,135.0,29,under,under,-3,3,home,push,home
3,2022-11-06,Arizona Cardinals,Seattle Seahawks,21,31,1.52,2.67,-3.5,3.5,49.0,1.76,2.10,-2.0,2.0,49.0,-192.0,167.0,-132.0,110.0,52,over,over,10,-10,away,away,away
4,2022-11-06,Atlanta Falcons,Los Angeles Chargers,17,20,2.40,1.62,3.0,-3.0,47.0,2.25,1.68,2.5,-2.5,49.5,140.0,-161.0,125.0,-147.0,37,under,under,3,-3,push,away,away
5,2022-11-06,Chicago Bears,Miami Dolphins,32,35,2.55,1.57,3.5,-3.5,41.5,2.65,1.54,4.0,-4.0,46.0,155.0,-175.0,165.0,-185.0,67,over,over,3,-3,home,home,away
6,2022-11-06,Cincinnati Bengals,Carolina Panthers,42,21,1.22,4.50,-9.0,9.0,44.0,1.29,3.70,-7.0,7.0,42.5,-455.0,350.0,-345.0,270.0,63,over,over,-21,21,home,home,home
7,2022-11-06,Detroit Lions,Green Bay Packers,15,9,2.55,1.57,3.5,-3.5,49.0,2.67,1.52,4.0,-4.0,49.5,155.0,-175.0,167.0,-192.0,24,under,under,-6,6,home,home,home
8,2022-11-06,Jacksonville Jaguars,Las Vegas Raiders,27,20,2.05,1.80,1.5,-1.5,47.0,2.20,1.71,2.5,-2.5,48.0,105.0,-125.0,120.0,-141.0,47,push,under,-7,7,home,home,home
9,2022-11-06,New England Patriots,Indianapolis Colts,26,3,1.36,3.25,-6.0,6.0,41.0,1.43,2.90,-5.0,5.0,40.0,-278.0,225.0,-233.0,190.0,29,under,under,-23,23,home,home,home


In [10]:
# Rename Columns

renamed_betting_df = betting_data.copy()

renamed_betting_df = renamed_betting_df.rename(columns = {
    'Date': 'date',
    'Home Team': 'home_team',
    'Away Team': 'away_team',
    'Home Score': 'home_score',
    'Away Score': 'away_score',
    'Home Odds Open': 'home_odds_open',
    'Away Odds Open': 'away_odds_open',
    'Home Line Open': 'home_spread_open',
    'Away Line Open': 'away_spread_open',
    'Total Score Open': 'total_score_open',
    'Home Odds Close': 'home_odds_close',
    'Away Odds Close': 'away_odds_close',
    'Home Line Close': 'home_spread_close',
    'Away Line Close': 'away_spread_close',
    'Total Score Close': 'total_score_close',
    'Home ML Open': 'home_ml_open',
    'Away ML Open': 'away_ml_open',
    'Home ML Close': 'home_ml_close',
    'Away ML Close': 'away_ml_close',
    'Total Score': 'total_score',
    'Over/Under Open Result': 'over_under_open_result',
    'Over/Under Close Result': 'over_under_close_result',
    'Home Team Actual Line': 'home_team_actual_line',
    'Away Team Actual Line': 'away_team_actual_line',
    'Spread Cover Open Result': 'spread_cover_open_result',
    'Spread Cover Close Result': 'spread_cover_close_result',
    'Game Winner': 'game_winner'
})

renamed_betting_df.head()

,date,home_team,away_team,home_score,away_score,home_odds_open,away_odds_open,home_spread_open,away_spread_open,total_score_open,home_odds_close,away_odds_close,home_spread_close,away_spread_close,total_score_close,home_ml_open,away_ml_open,home_ml_close,away_ml_close,total_score,over_under_open_result,over_under_close_result,home_team_actual_line,away_team_actual_line,spread_cover_open_result,spread_cover_close_result,game_winner
0,2022-11-07,New Orleans Saints,Baltimore Ravens,13,27,2.45,1.60,3.5,-3.5,49.0,2.10,1.76,1.5,-1.5,46.0,145.0,-167.0,110.0,-132.0,40,under,under,14,-14,away,away,away
1,2022-11-06,Kansas City Chiefs,Tennessee Titans,20,17,1.18,5.25,-10.5,10.5,47.0,1.11,6.75,-14.0,14.0,45.0,-556.0,425.0,-909.0,575.0,37,under,under,-3,3,away,away,home
2,2022-11-06,Tampa Bay Buccaneers,Los Angeles Rams,16,13,1.80,2.05,-1.5,1.5,43.0,1.64,2.35,-3.0,3.0,42.5,-125.0,105.0,-156.0,135.0,29,under,under,-3,3,home,push,home
3,2022-11-06,Arizona Cardinals,Seattle Seahawks,21,31,1.52,2.67,-3.5,3.5,49.0,1.76,2.10,-2.0,2.0,49.0,-192.0,167.0,-132.0,110.0,52,over,over,10,-10,away,away,away
4,2022-11-06,Atlanta Falcons,Los Angeles Chargers,17,20,2.40,1.62,3.0,-3.0,47.0,2.25,1.68,2.5,-2.5,49.5,140.0,-161.0,125.0,-147.0,37,under,under,3,-3,push,away,away


In [15]:
# Truncated betting df with opening lines

opening_betting = renamed_betting_df.copy()
opening_betting = opening_betting[['date', 'home_team', 'away_team', 'home_score', 'away_score', 'total_score', 
                                  'home_ml_open', 'away_ml_open', 'home_spread_open', 'away_spread_open',
                                  'total_score_open', 'over_under_open_result', 'home_team_actual_line',
                                  'away_team_actual_line', 'spread_cover_open_result', 'game_winner']]

opening_betting = opening_betting.rename(columns = {
    'home_ml_open': 'home_moneyline',
    'away_ml_open': 'away_moneyline',
    'home_spread_open': 'home_spread',
    'away_spread_open': 'away_spread',
    'total_score_open': 'total_score',
    'over_under_open_result': 'over_under_result',
    'spread_cover_open_result': 'spread_cover_result'
})

opening_betting.to_csv('../data/truncated_cleaned_betting_data_opening_lines.csv')
opening_betting.head()

,date,home_team,away_team,home_score,away_score,total_score,home_moneyline,away_moneyline,home_spread,away_spread,total_score,over_under_result,home_team_actual_line,away_team_actual_line,spread_cover_result,game_winner
0,2022-11-07,New Orleans Saints,Baltimore Ravens,13,27,40,145.0,-167.0,3.5,-3.5,49.0,under,14,-14,away,away
1,2022-11-06,Kansas City Chiefs,Tennessee Titans,20,17,37,-556.0,425.0,-10.5,10.5,47.0,under,-3,3,away,home
2,2022-11-06,Tampa Bay Buccaneers,Los Angeles Rams,16,13,29,-125.0,105.0,-1.5,1.5,43.0,under,-3,3,home,home
3,2022-11-06,Arizona Cardinals,Seattle Seahawks,21,31,52,-192.0,167.0,-3.5,3.5,49.0,over,10,-10,away,away
4,2022-11-06,Atlanta Falcons,Los Angeles Chargers,17,20,37,140.0,-161.0,3.0,-3.0,47.0,under,3,-3,push,away


In [16]:
# Truncated betting df with closing lines

closing_betting = renamed_betting_df.copy()
closing_betting = closing_betting[['date', 'home_team', 'away_team', 'home_score', 'away_score', 'total_score', 
                                  'home_ml_close', 'away_ml_close', 'home_spread_close', 'away_spread_close',
                                  'total_score_close', 'over_under_close_result', 'home_team_actual_line',
                                  'away_team_actual_line', 'spread_cover_close_result', 'game_winner']]

opening_betting = opening_betting.rename(columns = {
    'home_ml_close': 'home_moneyline',
    'away_ml_close': 'away_moneyline',
    'home_spread_close': 'home_spread',
    'away_spread_close': 'away_spread',
    'total_score_close': 'total_score',
    'over_under_close_result': 'over_under_result',
    'spread_cover_close_result': 'spread_cover_result'
})

opening_betting.to_csv('../data/truncated_cleaned_betting_data_closing_lines.csv')
opening_betting.head()

,date,home_team,away_team,home_score,away_score,total_score,home_moneyline,away_moneyline,home_spread,away_spread,total_score,over_under_result,home_team_actual_line,away_team_actual_line,spread_cover_result,game_winner
0,2022-11-07,New Orleans Saints,Baltimore Ravens,13,27,40,145.0,-167.0,3.5,-3.5,49.0,under,14,-14,away,away
1,2022-11-06,Kansas City Chiefs,Tennessee Titans,20,17,37,-556.0,425.0,-10.5,10.5,47.0,under,-3,3,away,home
2,2022-11-06,Tampa Bay Buccaneers,Los Angeles Rams,16,13,29,-125.0,105.0,-1.5,1.5,43.0,under,-3,3,home,home
3,2022-11-06,Arizona Cardinals,Seattle Seahawks,21,31,52,-192.0,167.0,-3.5,3.5,49.0,over,10,-10,away,away
4,2022-11-06,Atlanta Falcons,Los Angeles Chargers,17,20,37,140.0,-161.0,3.0,-3.0,47.0,under,3,-3,push,away
